In [1]:
from pyaniml import AnIMLDocument, Sample, Parameter, Category, ExperimentStep, Device, IndividualValueSet, Series, SeriesSet

# Creating an AnIML document

At the beginning an ```AnIMLDocument``` has to be initialized to construct a data report. This step is not mandatory, but can help preservee structure. One can alson start with individual elements and combine them at the end. However it is advised to first initialize and then building the rest.

In [2]:
animldoc = AnIMLDocument()

## Structure of an AnIML document

Three types of sets define an AnIML document:

* SampleSet: Documentation of all the samples that were part of the experiment.
* ExperimentStepSet: This container reflects all necessary processes and results that have been generated.
* AuditTrailEntrySet: I have no clue 😁

## Setting samples and their properties

Samples can be described in a very sparse or detiled way. AnIML does not set any boundaries here, but offers the opportunity to characterize via parameters or categories (container for multiple parameters)

In [3]:
# Initialize sample
sample_1 = Sample(id="S1", name="Sample1")

# Define some parameters
param_1 = Parameter(name="Param1", parameter_type="Int32", value=int(10))
param_2 = Parameter(name="Param2", parameter_type="String", value="Some kind of description")
param_3 = Parameter(name="Param3", parameter_type="String", value="I am not part of a category")

# Group these in a category if necessary
category = Category(name="Specific set of parameters that need to be grouped", parameters=[param_1, param_2])

# Finally, add these to the sample ...
sample_1.add_property(category)
sample_1.add_property(param_3)

# ... and the sample to the document
animldoc.add_sample(sample_1)

**Quick note**

Every object present in the AnIML document can be printed in a dataclass manner or as JSON/XML string. Generally, the library is built on top of the **SoftData** concept, where data models are abstract class models rather than specific formats. This has the consequence, that such a model can live in both JSON and XML formats without specific readers/writers.

In [4]:
# Inspect the sample!
print(sample_1.toXML())

<?xml version="1.0" encoding="UTF-8"?>
<Sample sampleID="S1" name="Sample1">
  <Parameter name="Param3" parameterType="String">
    <S>I am not part of a category</S>
  </Parameter>
  <Category name="Specific set of parameters that need to be grouped">
    <Parameter name="Param1" parameterType="Int32">
      <I>10</I>
    </Parameter>
    <Parameter name="Param2" parameterType="String">
      <S>Some kind of description</S>
    </Parameter>
  </Category>
</Sample>



## Setting experiment steps

Experiment steps are the core of any AnIML document, since they describe the process and results that were gathered in the scope of an experiment. Thus this type of class is much more complex and comprehends the following categories:

* Infrastructure: References the samples that have been processed in this step.
* Method: General information about the devices and methods used as well as the authors.
* Result: Composite class, which holds parameters, series as well as the grouped siblings.

In [5]:
# Initialize the first experiment step
exp_step_1 = ExperimentStep(name="First step", experiment_step_id="ES1")

### Infrastructure

In [6]:
# Add some infrastructure, lets use the sample we previously defined
exp_step_1.add_sample_reference(sample=sample_1, role="Shows a reference", sample_purpose="Educational")

### Methods

In [7]:
# Lets say we have a method, which has some parameters
method_param_1 = Parameter(name="Volume control", parameter_type="Int32", value=int(100))
method_param_2 = Parameter(name="Program", parameter_type="String", value="Full power!")
method_cat = Category(name="Device Setup", parameters=[method_param_1, method_param_2])

# ... and is a physical device
device = Device(name="Cool device", firmware_version="1.0", serial_number="126656563")

# Finally, add these to the method part
exp_step_1.add_method(method_cat)
exp_step_1.add_method(device)

### Results

In [8]:
# Now that methods are defined, lets add our results
# It behaves the same as methods but can also take (sets of) series
# as elements

series_1_values = IndividualValueSet([1.2,2.4,5.6])
series_1 = Series(
    name="Frequency", id="Freq1", data=series_1_values, data_type="Float32", dependency="dependent", plot_scale="none"
)

series_2_values = IndividualValueSet([20.2,22.4,53.6])
series_2 = Series(
    name="Amplitude", id="Amp1", data=series_2_values, data_type="Float32", dependency="independet", plot_scale="none"
)

series_set = SeriesSet(name="Sound output", series=[series_1, series_2])

# Finally, add it to the result section ...
exp_step_1.add_result(series_set)

# ... and the experiment step to the document
animldoc.add_experiment_step(exp_step_1)

# Finished!

The AnIML document is completely defined and no other steps required other than to call either one of the supported formats to which SoftData can be exported:

* XML
* JSON
* Python Dict

In [9]:
# Lets look at the XML
xml_string = animldoc.toXML()
print(xml_string)

<?xml version="1.0" encoding="UTF-8"?>
<AnIML>
  <SampleSet>
    <Sample sampleID="S1" name="Sample1">
      <Parameter name="Param3" parameterType="String">
        <S>I am not part of a category</S>
      </Parameter>
      <Category name="Specific set of parameters that need to be grouped">
        <Parameter name="Param1" parameterType="Int32">
          <I>10</I>
        </Parameter>
        <Parameter name="Param2" parameterType="String">
          <S>Some kind of description</S>
        </Parameter>
      </Category>
    </Sample>
  </SampleSet>
  <ExperimentStepSet>
    <ExperimentStep xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" name="First step" experimentStepID="ES1" xsi:type="ExperimentStep">
      <Infrastructure>
        <SampleReferenceSet>
          <SampleReference sampleID="S1" role="Shows a reference" samplePurpose="Educational"/>
        </SampleReferenceSet>
      </Infrastructure>
      <Method>
        <Device>
          <Name>Cool device</Name>
        